In [5]:
import cv2 #To extract images
from sklearn.externals import joblib #To import saved knn model
import numpy as np #to play with numpy arrays
from PIL import Image,ImageTk #image conversion
from tkinter import * #to create application 
import os #to change current working directory
import pandas as pd #to convert into dataframe
from tkinter import filedialog #To create browse option


root = Tk() #Creating application
root.geometry("1500x1000") #Application width
#Heading of the application
heading = Label(root, text = "Welcome to the Digit Recognition application",font =("arial",20,"bold"),fg= "black" ).pack()

label1 = Label(root,text="Select the Image: ",font =("arial",12,"bold"),fg= "black").pack()
path = ""

#Creating browse option
def browsefunc():
    filename = filedialog.askopenfilename()
    pathlabel.config(text=filename)
    global path
    path = str(filename)
    
#Creating browse buttin
browsebutton = Button(root, text="Browse", command=browsefunc)
browsebutton.pack()

pathlabel = Label(root)
pathlabel.pack()





def do_it():
    
    # Read the input image 
    
    impath = path 
    #Reading the image
    im = cv2.imread(impath,1)
    #Convert the image into grayscale
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)

    
    # Threshold the image
    ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY_INV)

    # Find contours in the image
    img,ctrs, hier = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get rectangles contains each contour
    rects = [cv2.boundingRect(ctr) for ctr in ctrs]
    #Creating Empty lists
    digits = list()
    lst = list()
    pred = list()
    # Load saved knn model from file
    joblib_file = "C:/Users/Sanjeev Varma/Desktop/Capstone/Signature Recognition/Data/joblib_model.pkl"
    joblib_model = joblib.load(joblib_file)

    for rect in rects:
        # Draw the rectangles
        cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3) 
        # Make the rectangular region around the digit
        leng = int(rect[3] * 1.6)
        pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
        pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
        roi = im_th[pt1:pt1+leng, pt2:pt2+leng]
        # Resize the image
        roi = cv2.resize(roi, (40, 40), interpolation=cv2.INTER_AREA)
        roi = cv2.dilate(roi, (5, 5))
        #Get the pixel values
        pil_im = Image.fromarray(roi)
        data = list(pil_im.getdata())
        for x in data:
            digits.append(x)
        lst.append(digits)
        digits = []
        data = []
        df = pd.DataFrame(lst)
        lst = []
        #Sending the values as input the model
        predict = joblib_model.predict(df)
        pred.append(list(predict))
    #To show the image in the application window    
    digpath = path
    picture = Image.open(digpath)
    picture = picture.resize((600, 300), Image.ANTIALIAS)
    pic = ImageTk.PhotoImage(picture)
    label2 = Label(root,text="Your Image is: ",font =("arial",12,"bold"),fg= "black").pack()
    label3=Label(root,image = pic)
    label3.image = pic
    label3.pack()
    #Printing the predicted Values
    label4 = Label(root,text = pred,font =("arial",12,"bold"),fg= "red").pack()  
    

submit = Button(root, text="Submit",width = 10,height = 1,bg="lightgray",command = do_it).pack()
mainloop( )